<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Mounted at /content/drive


In [ ]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-04-30 20:37:16
-------------------
Error fetching data for 3MINDIA: Too Many Requests. Rate limited. Try after a while.
Error fetching data for 5PAISA: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AARTIIND: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AAVAS: Too Many Requests. Rate limited. Try after a while.
Error fetching data for ABB: Too Many Requests. Rate limited. Try after a while.
Error fetching data for ABBOTINDIA: Too Many Requests. Rate limited. Try after a while.
Error fetching data for ABSLAMC: Too Many Requests. Rate limited. Try after a while.
Error fetching data for ACC: Too Many Requests. Rate limited. Try after a while.
Error fetching data for ADANIPORTS: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AETHER: Too Many Requests. Rate limited. Try after a while.
Error fetching data for AKZOINDIA: Too Many Requests. Rate limited. Try after a while.
Error f

In [ ]:
# X-LC and H-LC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].isin(['X-LC','H-LC'])]

df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)
df_tmp['Rnk_ATH%'] = df_tmp['ATH%'].rank(method='max', ascending=False)
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Total_Rank'] = df_tmp['Rnk_Dev%_200'] + df_tmp['Rnk_Dev%_PE'] + df_tmp['Rnk_Gained%'] + df_tmp['Rnk_ATH%'] + df_tmp['Rnk_Sales_Grwth%'] + df_tmp['Rnk_Profit_Grwth%'] + df_tmp['Rnk_MCap']
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'FTT%', 'ATH%','Gained%', 'Total_Rank', 'Strategy', 'Category', 'InFolio']
data_table.DataTable(df_tmp[cols].sort_values(by = 'Total_Rank', ascending=True), include_index=False)

In [ ]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

In [ ]:
# watchlist
df_tmp = df_prospects_features[df_prospects_features['InFolio'] == -1].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

In [ ]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

In [ ]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

In [ ]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

In [ ]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

In [ ]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)